In [ ]:
# Add starting and ending times to functions to optimize order.
# optimised for 2019 not sure if it work for all the other years

import pandas as pd
from datetime import datetime
import os


# 1 Drop unimportant columns

def function_1(year):
    
    chunksize = 1000000
    i = 0
    header = True
    if (os.path.isfile(year + '_1_columndelete.csv')):
        os.remove(year + '_1_columndelete.csv')


    for chunk in pd.read_csv(year + '_Yellow_Taxi_Trip_Data.csv', chunksize = chunksize):
        i = i+1
        print(i)
        #drop vendorID, passenger_count, extra, payment_type ..
        chunk = chunk.drop(columns=['VendorID','passenger_count','RatecodeID','store_and_fwd_flag','payment_type','fare_amount', 'extra','mta_tax','tolls_amount','improvement_surcharge','congestion_surcharge'])
        chunk.to_csv(year + '_1_columndelete.csv', mode = 'a', header = header)
        header = False

    print('finished 1')




# 2a Compare Datetime stepbystep, check start before 6am (2019)
def function_2a(year):
    if (os.path.isfile(year + '_2a_startbefore6am.csv')):
        os.remove(year + '_2a_startbefore6am.csv')
    chunksize = 1000
    i = 0
    header = True
    compare_am = datetime.strptime('06:00:00 AM', '%I:%M:%S %p').time()

    for chunk in pd.read_csv(year + '_1_columndelete.csv', chunksize = chunksize, index_col=0):
        counter = 0
        print(i)
        while (counter < chunksize):
            string_pickup = chunk['tpep_pickup_datetime'][i]
            #Hier kann man eine Zeile weglassen durch direkt time()
            datetime_pickup = datetime.strptime(string_pickup,'%m/%d/%Y %I:%M:%S %p')
            help_time_start = datetime_pickup.time()

            # Check if time is in time-window
            if (compare_am > help_time_start):        
                chunk = chunk.drop([i])                            
            i =  i + 1
            counter = counter + 1
        chunk.to_csv(year + '_2a_startbefore6am.csv', mode = 'a', header = header)
        header = False
    print('finished_2a')

        
# 2b Compare Datetime stepbystep, check start after 6pm (2019)
def function_2b(year):        
    if (os.path.isfile(year + '_2b_startafter6pm.csv')):
        os.remove(year + '_2b_startafter6pm.csv')
    chunksize = 1000
    i = 0
    header = True

    compare_pm = datetime.strptime('06:00:00 PM', '%I:%M:%S %p').time()

    for chunk in pd.read_csv(year + '_2a_startbefore6am.csv', chunksize = chunksize):
        counter = 0
        print(i)
        while (counter < chunksize):
            string_pickup = chunk['tpep_pickup_datetime'][i]
            help_time_start = datetime.strptime(string_pickup,'%m/%d/%Y %I:%M:%S %p').time()

            # Check if time is in time-window
            if (compare_pm < help_time_start):        
                chunk = chunk.drop([i])                            
            i =  i + 1
            counter = counter + 1
        chunk.to_csv(year + '_2b_startafter6pm.csv', mode = 'a', header = header)
        header = False
    print('finished 2b')    

                     
# 2c Compare Datetime stepbystep, check end before 6am (2019)
def function_2c(year):
    if (os.path.isfile(year + '_2c_endbefore6am.csv')):
        os.remove(year + '_2c_endbefore6am.csv')
    chunksize = 1000
    i = 0
    header = True

    compare_am = datetime.strptime('06:00:00 AM', '%I:%M:%S %p').time()

    for chunk in pd.read_csv(year + '_2b_startafter6pm.csv', chunksize = chunksize, index_col=0):
        counter = 0
        print(i)
        while (counter < chunksize):
            string_dropoff = chunk['tpep_dropoff_datetime'][i]
            help_time_end = datetime.strptime(string_dropoff,'%m/%d/%Y %I:%M:%S %p').time()

            # Check if time is in time-window
            if (compare_am > help_time_end):        
                chunk = chunk.drop([i])                

            i =  i + 1
            counter = counter + 1
        chunk.to_csv(year + '_2c_endbefore6am.csv', mode = 'a', header = header)
        header = False
    print('finished 2c')

                     
# 2d Compare Datetime stepbystep, check end after 6pm (2019)
                     
def function_2d(year):
    if (os.path.isfile(year + '_2d_endafter6pm.csv')):
        os.remove(year + '_2d_endafter6pm.csv')
    chunksize = 1000
    i = 0
    header = True

    compare_pm = datetime.strptime('06:00:00 PM', '%I:%M:%S %p').time()

    for chunk in pd.read_csv('2019_2c_endbefore6am.csv', chunksize = chunksize, index_col=0):
        counter = 0
        print(i)
        while (counter < chunksize):
            string_pickup = chunk['tpep_dropoff_datetime'][i]
            help_time_end = datetime.strptime(string_dropoff,'%m/%d/%Y %I:%M:%S %p').time()

            # Check if time is in time-window
            if (compare_pm < help_time_end):        
                chunk = chunk.drop([i])                

            i =  i + 1
            counter = counter + 1
        chunk.to_csv(year + '_2d_endafter6pm.csv', mode = 'a', header = header)
        header = False
    print('finished 2d')

                     
                     
                     
# 3 Check if startzone == endzone, then delete row
                     
def function_3(year):
    if (os.path.isfile(year + '_3_deletesamezone.csv')):
        os.remove(year + '_3_deletesamezone.csv')
    chunksize = 1000
    i = 0
    header = True
    rowsdropped = 0

    compare_am = datetime.strptime('06:00:00', '%H:%M:%S').time()

    for chunk in pd.read_csv(year + '_2d_endafter6pm.csv', chunksize = chunksize, index_col=0):
        counter = 0
        print(i)
        while (counter < chunksize):
            startzone = chunk['PULocationID'][i]
            endzone = chunk['DOLocationID'][i]

            # Check if zones are equal
            if (startzone == endzone):        
                chunk = chunk.drop([i])
                rowsdroped += 1

            i += 1
            counter +=  1
        chunk.to_csv(year + '_3_deletesamezone.csv', mode = 'a', header = header)
        header = False
    print('finished 3')


# 4 Costs = Total costs - Tip, delete tip !!! not finished

def function_4(year):
    if (os.path.isfile(year + '_4_calculate_costs.csv')):
        os.remove(year + '_4_calculate_costs.csv')
    chunksize = 1000
    i = 0
    header = True

    for chunk in pd.read_csv(year + '_3_deletesamezone.csv', chunksize = chunksize, index_col=0):
        counter = 0
        print(i)
        while (counter < chunksize):
            total_amount = chunk['total_amount'][i]
            tip_amount = chunk['tip_amount'][i]
            
            # !!! could be necassary to cast from String to Float
            cost = total_amount - tip_amount
            
            # !!! add calculated value
            
            


            i += 1
            counter +=  1
        
        # Delete total_amount and tip_amount
        
        chunk = chunk.drop(columns=['total_amount', 'tip_amount'])

        
        
        chunk.to_csv(year + '_4_calculate_costs.csv', mode = 'a', header = header)
        header = False
    print('finished 4')

    
    
    
# 5 duration = timeend - timestart !!! not finished
    
def function_5(year):    
    
    if (os.path.isfile(year + '_5_add_duration.csv')):
        os.remove(year + '_5_add_duration.csv')
    
    header = True
    chunksize = 1000
    i = 0

    for chunk in pd.read_csv(year + '_4_calculate_costs.csv', chunksize = chunksize, index_col=0):

        #Get duration of trip
        counter = 0
        while (counter < chunksize):
            string_pickup = chunk['tpep_pickup_datetime'][i]
            datetime_pickup = datetime.strptime(string_pickup,'%Y-%m-%d %H:%M:%S')
            string_dropoff = chunk['tpep_dropoff_datetime'][i]
            datetime_dropoff = datetime.strptime(string_dropoff,'%Y-%m-%d %H:%M:%S')
            difference = datetime_dropoff - datetime_pickup
            # print(string_pickup, string_dropoff, 'Difference: ', difference)
            
            
            # !!! add time to data as new column
            
            
            
            i =  i + 1
            counter = counter + 1
            
        chunk.to_csv(year + '_5_add_duration', mode = 'a', header = header)
        header = False
    print('finished 5')            
            
            
#function_1('2019')    
#function_2a('2019')
function_2b('2019')
#function_2c('2019') 
#function_2d('2019')
#function_3('2019')
# function_4('2019') !!! not finished
# function_5('2019') !!! not finished

# !!! would be nice to shut down pc after completing all tasks



0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
